In [14]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [129]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/"
browser.visit(url)

#THERE ARE 24 PAGES OF DOG BREEDS
#https://www.akc.org/dog-breeds/page/24/

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [3]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/german-shepherd-dog/"
browser.visit(url)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
dog_breed = soup.find("h1").get_text()

In [6]:
temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()

In [7]:
for i in range(5):
    akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
    height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
    weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
    life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
    group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()

In [8]:
image_url = soup.find_all('img')[4]["data-src"]

In [9]:
description = soup.find("div", class_="breed-hero__footer").get_text()

In [10]:
for i in range(5):
    brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
    shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
    energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
    trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
    temperment = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')

In [124]:
table = soup.find_all('table')[0] # Grab the first table

In [127]:
color_list = []
final_color_list = []

for row in table.find_all('td'):
    if 'Description' in row:
        continue
    elif 'Standard Colors' in row:
        continue
    elif 'Registration Code' in row:
        continue
    elif "Check Mark For Standard Color" in row:
        continue
    else:
        color = row.get_text()
        color_list.append(color)

for i in color_list:
    if "Check Mark For Standard Color" in i:
        continue     
    else:
        try:
            if int(i) > 0:
                continue
        except:
            final_color_list.append(i)

In [128]:
final_color_list

['Bi-Color',
 'Black',
 'Black & Cream',
 'Black & Red',
 'Black & Silver',
 'Black & Tan',
 'Blue',
 'Gray',
 'Liver',
 'Sable',
 'White']